1. 20 
2. 20
3. **a.10 b.10** c.10 d.10 e.20

All these details can be found on Scientia. 

Your file should be "Report.pdf". 

You should not submit your code with your coursework. 

We will not check the code. 

Please submit only your results, comments and generated images. 

You can use any library for your implementation. 

To answer questions 3a-d, it is not necessary to rectify your images. However, if you choose to do it, please state it clearly. To find the salient points which meet the epipolar constraint, you just need to identify which pairs of matches satisfy the following condition (you can relax it by setting a low threshold instead of zero): 

(X′)^T*F*X = 0
 
For question 3e, one way to estimate the area of the swimming pool and the length of the football field, is to find the 3D coordinates of their vertices. In that case, rectification should be applied to estimate disparity and then the 3D coordinates (X,Y,Z) for each vertex.

Please describe the steps that you followed to estimate the pool area and the length of the football field. This will give you marks even if your final result is not correct.

No need to use an appendix. Just add all your results in your report.


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [ ]:
norm = False
do_plot = [False, False, False, False, True]

w = 1920    # 图像宽
h = 1080    # 图像长

nfeatures = 0               # 最大关键点数量
edgeThreshold = 3.1         # 过滤边缘响应的阈值(大于的剔除)
contrastThreshold = 0.056   # 过滤低对比度关键点的阈值(小于的剔除)
match_method = 'nndr'       # match method ['nndr', 'nn', 'cc']
ratio_thresh = 0.7          # nndr比值阈值(大于的剔除)

draw_match = -1
flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS  # cv2.DRAW_MATCHES_FLAGS_DEFAULT


In [ ]:
norm_T = np.array([[2/w, 0, -1],[0, 2/h, -1],[0, 0, 1],])

intrinsic_param_1 = np.array(
    [[1.600e+03, 0.000e+00, 9.595e+02], 
     [0.000e+00, 1.600e+03, 5.395e+02], 
     [0.000e+00, 0.000e+00, 1.000e+00]] 
)

extrinsic_param_1 = np.array(
    [[-6.32422984e-01, -7.74574101e-01, 8.72639567e-03, -2.36433081e+00], 
     [-5.00836670e-01, 4.00276423e-01, -7.67425179e-01, -1.74806440e+00], 
     [ 5.90934694e-01, -4.89707828e-01, -6.41079128e-01, 2.59576015e+01], 
     [ 0., 0., 0., 1.]]
)

intrinsic_param_2 = np.array(
    [[1.49333333e+03, 0.00000000e+00, 9.78700000e+02], 
     [0.00000000e+00, 1.49333333e+03, 5.20300000e+02], 
     [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]
)

extrinsic_param_2 = np.array(
    [[-0.5845883, -0.81050003, -0.03669427, -2.39520617], 
     [-0.5041514, 0.39832053, -0.76627171, -1.79913743], 
     [ 0.6356793, -0.42945388, -0.64146805, 26.26804151], 
     [ 0., 0., 0., 1.]]
)

distortion_1 = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

distortion_2 = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

# 0.keypoint 标准化函数
def normalize_points(pts):
    if norm:
        points_h = np.hstack([pts, np.ones((pts.shape[0], 1))])
        normalized_points_h = (norm_T @ points_h.T).T
        norm_points = normalized_points_h[:, :2]
        return norm_points
    return pts

# 0.fundamental matrix 逆标准化函数
def denormalize_FM(FM):
    if norm:
        return norm_T.T @ FM @ norm_T
    return FM

# 0.fundamental matrix 计算函数
def compute_fundamental_matrix(intrinsic_param_1, extrinsic_param_1, intrinsic_param_2, extrinsic_param_2):
    # Extract intrinsic matrices
    K1 = intrinsic_param_1
    K2 = intrinsic_param_2

    # Extract extrinsic parameters
    R1 = extrinsic_param_1[:3, :3]
    t1 = extrinsic_param_1[:3, 3]
    R2 = extrinsic_param_2[:3, :3]
    t2 = extrinsic_param_2[:3, 3]

    # Compute relative pose
    R = R2 @ R1.T  # Relative rotation
    t = t2 - R @ t1  # Relative translation
    # print(f"R_vec: {R}")  #一样
    # print(f"T_vec: {t}")  #一样

    # Skew-symmetric matrix for t
    t_skew = np.array([
        [0, -t[2], t[1]],
        [t[2], 0, -t[0]],
        [-t[1], t[0], 0]
    ])

    # Compute the Fundamental Matrix
    F = np.linalg.inv(K2).T @ t_skew @ R @ np.linalg.inv(K1)
    
    return F, R, t

# 0.epipolar constraint 残差计算函数
def check_epipolar_constraint(fundamental_matrix, pts1, pts2):
    # Convert points to homogeneous coordinates
    pts1_homogeneous = np.hstack([pts1, np.ones((pts1.shape[0], 1))])  # Nx3
    pts2_homogeneous = np.hstack([pts2, np.ones((pts2.shape[0], 1))])  # Nx3

    # Compute the epipolar constraint for each pair of points
    residuals = []
    for p1, p2 in zip(pts1_homogeneous, pts2_homogeneous):
        # Compute the epipolar constraint: p2^T * F * p1
        residual = np.dot(np.dot(p2, fundamental_matrix), p1)
        residuals.append(abs(residual))  # Use absolute value for easy interpretation
    return residuals

# 0. Stereo image rectification
def compute_disparity_and_rectification(frame1, frame2, intrinsic1, distortion1, intrinsic2, distortion2, R, T):
    # Image dimensions
    h, w = frame1.shape[:2]

    # Stereo rectification transforms
    R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
        intrinsic1, distortion1, intrinsic2, distortion2, (w, h), R, T,
        flags=cv2.CALIB_ZERO_DISPARITY, alpha=1
    )

    # Compute rectification maps
    map1x, map1y = cv2.initUndistortRectifyMap(
        intrinsic1, distortion1, R1, P1, (w, h), cv2.CV_32FC1
    )
    map2x, map2y = cv2.initUndistortRectifyMap(
        intrinsic2, distortion2, R2, P2, (w, h), cv2.CV_32FC1
    )

    # Apply rectification
    rectified1 = cv2.remap(frame1, map1x, map1y, cv2.INTER_LINEAR)
    rectified2 = cv2.remap(frame2, map2x, map2y, cv2.INTER_LINEAR)

    # Compute disparity using StereoBM
    stereo = cv2.StereoBM_create(numDisparities=64, blockSize=15)
    disparity = stereo.compute(rectified1, rectified2).astype(np.float32) / 16.0

    return rectified1, rectified2, disparity

# 1. Plot the detected features on the provided pair of frames
def draw_keypoints(frame, keypoints):
    frame_color = cv2.merge((frame,frame,frame))

    for kp in keypoints:
        # 获取关键点的圆心位置
        x = int(kp.pt[0])
        y = int(kp.pt[1])
        random_color = tuple(np.random.randint(0, 255, size=3).tolist())

        if flags == cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS:
            # 获取关键点的大小作为半径
            radius = int(kp.size / 2)  # Scale is proportional to size
            # 获取带方向的关键点
            angle = np.deg2rad(kp.angle)  # 方向角转换为弧度
            length = radius  # 方向线长度
            x2 = int(x + length * np.cos(angle))
            y2 = int(y + length * np.sin(angle))

            # 在图像上绘制圆形表示关键点及其大小
            cv2.circle(frame_color, (x, y), radius=4, color=random_color, thickness=-1)
            cv2.circle(frame_color, (x, y), radius=radius, color=random_color, thickness=2)
            if type(x2) is int:
                cv2.line(frame_color, (x, y), (x2, y2), color=random_color, thickness=2)
            else:
                cv2.line(frame_color, (x, y), (x2[0], y2[0]), color=random_color, thickness=2)
                for _x2, _y2 in zip(x2[1:],y2[1:]):
                    random_color = tuple(np.random.randint(0, 255, size=3).tolist())
                    cv2.line(frame_color, (x, y), (_x2, _y2), color=random_color, thickness=2)
        else:
            cv2.circle(frame_color, (x, y), radius=4, color=random_color, thickness=-1)

    plt.figure(figsize=(15, 10))
    plt.imshow(frame_color)
    plt.axis("off")
    plt.show()

# 2. Create a composite image for 2 frames (centered overlay image)
def plot_match_centered_overlay(pts1, pts2, frame1, frame2):
    frame1_color = (173/255, 216/255, 230/255)
    frame2_color = (255/255, 182/255, 193/255)
    
    frame1_blue = cv2.merge(((frame1*frame1_color[0]).astype(np.uint8),(frame1*frame1_color[1]).astype(np.uint8),(frame1*frame1_color[2]).astype(np.uint8)))
    frame2_red = cv2.merge(((frame2*frame2_color[0]).astype(np.uint8),(frame2*frame2_color[1]).astype(np.uint8),(frame2*frame2_color[2]).astype(np.uint8)))

    # Manually draw matches on the composite image
    composite_with_matches = cv2.addWeighted(frame1_blue, 0.8, frame2_red, 0.8, 0)
    for pt1, pt2 in zip(pts1[:draw_match],pts2[:draw_match]):
        pt1 = tuple(map(int, pt1))
        pt2 = tuple(map(int, pt2))
        cv2.circle(composite_with_matches, pt1, 8, (0, 255, 0), -1)     # keypoint1-green
        cv2.circle(composite_with_matches, pt2, 8, (255, 0, 0), -1)     # keypoint2-red
        cv2.line(composite_with_matches, pt1, pt2, (255, 255, 0), 2)

    # Display the centered overlay with matches
    plt.figure(figsize=(15, 10))
    plt.imshow(composite_with_matches)
    plt.axis("off")
    legend_handles = [
        Line2D([0], [0], marker='o', color='none', label='Keypoints in Frame1', markerfacecolor=(0.,1.,0.), markersize=12),    #green
        Line2D([0], [0], marker='o', color='none', label='Keypoints in Frame2', markerfacecolor=(1.,0.,0.), markersize=12),    #red
    ]
    plt.legend(handles=legend_handles, loc='lower right', fontsize='large')
    plt.show()

# 3. Create a composite image for 2 frames (side-by-side image)
def plot_match_side_by_side(pts1, pts2, frame1, frame2):
    height1, width1 = frame1.shape
    height2, width2 = frame2.shape
    canvas = np.zeros((max(height1, height2), width1 + width2, 3), dtype=np.uint8)

    # Place the colored images side by side on the canvas
    canvas[:height1, :width1] = cv2.merge((frame1, frame1, frame1))
    canvas[:height2, width1:] = cv2.merge((frame2, frame2, frame2))

    # Manually draw matches on the composite image
    for pt1, pt2 in zip(pts1[:draw_match],pts2[:draw_match]):
        pt1 = tuple(map(int, pt1))
        pt2 = tuple(map(int, pt2))
        pt2 = (pt2[0] + width1, pt2[1]) #Offset pt2 coordinates because img2 is placed to the right of img1
        
        cv2.line(canvas, pt1, pt2, (225,225,0), thickness=2)
        cv2.circle(canvas, pt1, radius=10, color=(0,225,0), thickness=-1)
        cv2.circle(canvas, pt2, radius=10, color=(225,0,0), thickness=-1)

    # Display the centered overlay with matches
    plt.figure(figsize=(30, 10))
    plt.imshow(canvas)
    plt.axis("off")
    legend_handles = [
        Line2D([0], [0], marker='o', color='none', label='Keypoints in Frame1', markerfacecolor=(0.,1.,0.), markersize=12),    #green
        Line2D([0], [0], marker='o', color='none', label='Keypoints in Frame2', markerfacecolor=(1.,0.,0.), markersize=12),    #red
    ]
    plt.legend(handles=legend_handles, loc='lower right', fontsize='large')
    plt.show()


In [ ]:
""" Load images and get keypoints/descriptors """
# Load the uploaded images
frame1_path = "./Frame1.png"
frame2_path = "./Frame2.png"
ori_frame1 = cv2.imread(frame1_path)
ori_frame2 = cv2.imread(frame2_path)

# Convert images to grayscale
gray_frame1 = cv2.cvtColor(ori_frame1, cv2.COLOR_BGR2GRAY)
gray_frame2 = cv2.cvtColor(ori_frame2, cv2.COLOR_BGR2GRAY)

frame1 = gray_frame1
frame2 = gray_frame2

# Initialize SIFT detector
sift = cv2.SIFT_create(nfeatures = nfeatures, contrastThreshold = contrastThreshold, edgeThreshold = edgeThreshold,)

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = sift.detectAndCompute(frame1, None)
keypoints2, descriptors2 = sift.detectAndCompute(frame2, None)

print(f"Number of keypoints in frame1: {len(keypoints1)}")
print(f"Number of keypoints in frame2: {len(keypoints2)}")

In [ ]:
""" Match the detected salient features """
if match_method == 'nndr':
    bf = cv2.BFMatcher(cv2.NORM_L2)
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_thresh * n.distance:  # 比值测试
            good_matches.append(m)
    matches = sorted(good_matches, key=lambda x: x.distance)
elif match_method == 'nn':
    bf = cv2.BFMatcher(cv2.NORM_L2)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)
elif match_method == 'cc':
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)

print(f"({match_method.upper()}) Number of matches: {len(matches)}")

In [ ]:
""" Estimate the fundamental matrix (using matched features) """
# Convert keypoints to numpy arrays
pts1 = normalize_points(np.float32([keypoints1[m.queryIdx].pt for m in matches]))
pts2 = normalize_points(np.float32([keypoints2[m.trainIdx].pt for m in matches]))

# Estimate the fundamental matrix using RANSAC to handle outliers
ransec_threshold = 20/1920 if norm else 20
F_match_ori, mask = cv2.findFundamentalMat(
    pts1, pts2, method=cv2.FM_RANSAC,
    ransacReprojThreshold = ransec_threshold,
    confidence = 0.999999,
    maxIters = 10000
)

# Filter inliers using the mask
inliers_pts1 = pts1[mask.ravel() == 1]
inliers_pts2 = pts2[mask.ravel() == 1]
print(f"Number of inliers: {len(inliers_pts1)} / {len(pts1)}")

F_match_ori = denormalize_FM(F_match_ori)
print("\nEstimated Fundamental Matrix (using matches):\n", F_match_ori)

# Refine the Fundamental Matrix using only inliers
F_match, mask_refined = cv2.findFundamentalMat(inliers_pts1, inliers_pts2, method=cv2.FM_8POINT)
F_match = denormalize_FM(F_match)
print("\nRefined Fundamental Matrix (using matches):\n", F_match)


In [ ]:
""" Estimate the fundamental matrix (using camera parameters) """
F_param, _, _ = compute_fundamental_matrix(intrinsic_param_1, extrinsic_param_1, intrinsic_param_2, extrinsic_param_2)
print("Fundamental Matrix (using params):\n", F_param)


In [ ]:
""" Verify the epipolar constraint for a set of matching keypoints """
pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches])
pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches])

# Check epipolar constraint
residuals_match = np.array(check_epipolar_constraint(F_match, pts1, pts2))
residuals_param = np.array(check_epipolar_constraint(F_param, pts1, pts2))
residuals_match_ori = np.array(check_epipolar_constraint(F_match_ori, pts1, pts2))

# Output results
threshold = 0.01
print(f"|       |{'':9}mean{'':8}|{'':9}std{'':9}|{'':9}var{'':9}|{'':9}min{'':9}|{'':9}max{'':9}| inliers")
for data, name in zip([residuals_param,residuals_match_ori,residuals_match,],['param','m_ori','match',]):
    inliers = [i for i, r in enumerate(data) if r < threshold]
    print(f"| {name} | {data.mean():.17f} | {data.std():.17f} | {data.var():.17f} | {data.min():.17f} | {data.max():19.16f} | {len(inliers):2} / {len(data):2}")

# for i, (residual_match,residual_param) in enumerate(zip(residuals_match,residuals_param)):
#     print(f"Point Pair {(i + 1):3}: {residual_match:20.17f} | {residual_param:20.17f} | {abs(residual_match-residual_param):20.17f}")


In [ ]:
""" Select Corner Points in Rectified Frame manually """
import pickle as pk

# 0.Compute rectification and disparity
_, R, T = compute_fundamental_matrix(intrinsic_param_1, extrinsic_param_1, intrinsic_param_2, extrinsic_param_2)
rectified1, rectified2, disparity_map = compute_disparity_and_rectification(frame1, frame2, intrinsic_param_1, distortion_1, intrinsic_param_2, distortion_2, R, T)

pk.dump(rectified1,open('./data/rectified1.pkl','wb'))
pk.dump(rectified2,open('./data/rectified2.pkl','wb'))
pk.dump(disparity_map,open('./data/disparity_map.pkl','wb'))

In [ ]:
# 0.Set informations from frame2
focal_length = intrinsic_param_1[0,0]  # in pixels
baseline = np.linalg.norm(T)    # distance between the two cameras
cx = intrinsic_param_1[0,2]     # principal point coordinates
cy = intrinsic_param_1[1,2]     # principal point coordinates


# 1.Set Corner Points in Pixel Coordinates (in rectified frame1)
swimming_pool_corners_2D = np.array([[587, 621], [1185, 900]])
football_touchline_2D = np.array([[1135, 683], [1671, 370]])
# swimming_pool_corners_2D = np.array([[487, 620], [1014, 897]])    #2
# football_touchline_2D = np.array([[985, 700], [1565, 349]])       #2


# 2.Calculate Depth Using Disparity Map
def disparity_to_depth(disparity, focal_length, baseline):
    if disparity <= 0:
        return -1
    with np.errstate(divide='ignore', invalid='ignore'):
        depth = focal_length * baseline / disparity
    return depth
swimming_pool_corners_depth = [disparity_to_depth(disparity_map[int(v), int(u)], focal_length, baseline) for u, v in swimming_pool_corners_2D]
football_touchline_depth = [disparity_to_depth(disparity_map[int(v), int(u)], focal_length, baseline) for u, v in football_touchline_2D]


# 3.Reconstruct 3D Points
def reconstruct_3D(points_2D, depths, focal_length, cx, cy):
    points_3D = []
    uni_depth = np.max(depths)
    for (u, v), d in zip(points_2D, depths):
        if d > 0:
            Z = d
            X = (u - cx) * Z / focal_length
            Y = (v - cy) * Z / focal_length
            points_3D.append([X, Y, Z])
        else:
            points_3D.append([np.nan, np.nan, np.nan])  # Handle invalid points
    return np.array(points_3D)
swimming_pool_corners_3D = reconstruct_3D(swimming_pool_corners_2D, swimming_pool_corners_depth , focal_length, cx, cy)
football_touchline_3D = reconstruct_3D(football_touchline_2D, football_touchline_depth, focal_length, cx, cy)
print(swimming_pool_corners_3D)
print(football_touchline_3D)

# 4.Compute Area and Length
def compute_rectangle_area(point1_3D, point2_3D):
    # Extract coordinates
    x1, y1, z1 = point1_3D
    x2, y2, z2 = point2_3D

    # Length and width (assuming rectangle aligned to axes or known plane)
    length = abs(x2 - x1)
    width = abs(y2 - y1)

    # Compute the area
    area = length * width
    return area

def compute_length(point1_3D, point2_3D):
    return np.linalg.norm(point1_3D - point2_3D)

# Compute area of the swimming pool
pool_area = compute_rectangle_area(swimming_pool_corners_3D[0],swimming_pool_corners_3D[1])
print(f"Estimated swimming pool area: {pool_area:.2f} square meters")

# Compute length of the football field touchline
touchline_length = compute_length(football_touchline_3D[0], football_touchline_3D[1])
print(f"Estimated touchline length: {touchline_length:.2f} meters")

In [ ]:
""" !Plot figures"""
# !PLOT-1: Plot the detected features on the provided pair of frames
if do_plot[0]:
    for frame, keypoints in zip([frame1,frame2],[keypoints1,keypoints2]):
        draw_keypoints(frame, keypoints)
        
# !PLOT-2: (centered overlay image) Create a composite image for 2 frames
if do_plot[1] and frame1.shape == frame2.shape:
    pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches])
    plot_match_centered_overlay(pts1, pts2, frame1, frame2)

# !PLOT-3: (side-by-side image) Create a composite image for 2 frames
if do_plot[2]:
    pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches])
    plot_match_side_by_side(pts1, pts2, frame1, frame2)

# !PLOT-4: Plot matches that meet the epipolar constraint
if do_plot[3]:
    pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches])
    residuals_match = np.array(check_epipolar_constraint(F_match, pts1, pts2))
    residuals_param = np.array(check_epipolar_constraint(F_param, pts1, pts2))
    residuals_match_ori = np.array(check_epipolar_constraint(F_match_ori, pts1, pts2))

    threshold = 0.01
    for data, name in zip([residuals_param,residuals_match_ori,residuals_match,],['F_param','F_match_ori','F_match',]):
        inliers = [i for i, r in enumerate(data) if r < threshold]
        print(f"Inliers for {name}: {len(inliers):2}/{len(data):2}")
        plot_match_centered_overlay(pts1[inliers], pts2[inliers], frame1, frame2)
        plot_match_side_by_side(pts1[inliers], pts2[inliers], frame1, frame2)


In [ ]:
""" !Optional:  Illustrate the disparity map and the rectification result """
def draw_horizontal_lines(image, line_spacing=150, color=(255, 200, 0)):
    image_with_lines = image.copy()
    h, w = image.shape[:2]
    for y in range(0, h, line_spacing):
        cv2.line(image_with_lines, (0, y), (w, y), color, thickness=2)
    return image_with_lines

if do_plot[4]:
    # Compute rectification and disparity
    _, R, T = compute_fundamental_matrix(intrinsic_param_1, extrinsic_param_1, intrinsic_param_2, extrinsic_param_2)
    rectified1, rectified2, disparity = compute_disparity_and_rectification(frame1, frame2, intrinsic_param_1, distortion_1, intrinsic_param_2, distortion_2, R, T)

    # Draw horizontal lines
    rectified1_color =  cv2.merge((rectified1, rectified1, rectified1))
    rectified2_color =  cv2.merge((rectified2, rectified2, rectified2))
    rectified1_with_lines = draw_horizontal_lines(rectified1_color)
    rectified2_with_lines = draw_horizontal_lines(rectified2_color)

    # Combine the images horizontally for visualization
    combined_image = np.hstack((rectified1_with_lines, rectified2_with_lines))

    # Display the result using matplotlib
    plt.figure(figsize=(30, 10))
    # plt.title("Rectified Images with Black Borders and Horizontal Lines")
    plt.imshow(combined_image)
    plt.axis('off')
    plt.show()

    plt.figure(figsize=(15, 5))
    # plt.title("Disparity Map")
    plt.imshow(disparity, cmap='jet')
    plt.colorbar()
    plt.axis("off")
    plt.show()
